# Packages

In [ ]:
from geopy.geocoders import Nominatim
import undetected_chromedriver as uc
from bs4 import BeautifulSoup
import requests
import random
import math
import time as t
import pandas as pd
import re
from deep_translator import GoogleTranslator
from tqdm.notebook import tqdm
from fake_useragent import UserAgent
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys

# Function

In [ ]:
def flatten(A):
    rt = []
    for i in A:
        if isinstance(i,list): rt.extend(flatten(i))
        else: rt.append(i)
    return rt

In [ ]:
def has_number(input_string):
    pattern = r"\d+"
    match = re.search(pattern, input_string)
    if match:
        return True
    else:
        return False

In [ ]:
def extract_number(input_string):
    pattern = r"(\d+)"
    match = re.search(pattern, input_string)
    if match:
        extracted_number = int(match.group(1))  # Convert the matched number to an integer
        return extracted_number
    else:
        return None 

In [ ]:
def addressToCoord(address):
    loc = Nominatim(user_agent="Geopy Library")
    getLoc = loc.geocode(address)
    return(getLoc.latitude, getLoc.longitude)

In [ ]:
def is_string_in_another_string_once(main_string, sub_string):
    index = main_string.find(sub_string)
    if index != -1:
        second_index = main_string.find(sub_string, index + 1)
        if second_index == -1:
            return True
    return False

In [ ]:
def uniquize(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

In [ ]:
def contains_only_numbers(input_string):
    pattern = r"^\d+$"
    return bool(re.match(pattern, input_string))

In [ ]:
ua = UserAgent()

# Scrape

In [ ]:
mother_url = 'https://www.kleinanzeigen.de'

In [ ]:
apartments = 'https://www.kleinanzeigen.de/s-wohnung-mieten/c203'
houses = 'https://www.kleinanzeigen.de/s-haus-mieten/c205'

## Pagination Generation

In [ ]:
header = {'User-Agent':str(ua.random)}
content = requests.get(apartments, headers = header)
soup = BeautifulSoup(content.content, 'html.parser')

cities_container = soup.find_all('ul', {'class':'treelist browsebox-itemlist is-root'})[-1]
cities_container_narrow = cities_container.find_all('li')
cities_apartments = [mother_url + i.find('a', {'href':True})['href'] for i in cities_container_narrow]

In [ ]:
header = {'User-Agent':str(ua.random)}
content = requests.get(houses, headers = header)
soup = BeautifulSoup(content.content, 'html.parser')

cities_container = soup.find_all('ul', {'class':'treelist browsebox-itemlist is-root'})[-1]
cities_container_narrow = cities_container.find_all('li')
cities_houses = [mother_url + i.find('a', {'href':True})['href'] for i in cities_container_narrow]

In [ ]:
all_pages_apartments = []
for city in tqdm(cities_apartments):
    link = city.split('/')
    link[-1:-1] = ['seite:50']
    link_last = '/'.join(link)

    header = {'User-Agent':str(ua.random)}
    content = requests.get(link_last, headers = header)
    
    true_last = content.url
    true_last_number = extract_number(true_last.split('/')[-2])
    pages = [true_last.replace(str(true_last_number), str(i)) for i in range(1, true_last_number + 1)]
    
    all_pages_apartments.append(pages)

all_pages_apartments = flatten(all_pages_apartments)

In [ ]:
all_pages_houses = []
for city in tqdm(cities_houses):
    link = city.split('/')
    link[-1:-1] = ['seite:50']
    link_last = '/'.join(link)

    header = {'User-Agent':str(ua.random)}
    content = requests.get(link_last, headers = header)
    
    true_last = content.url
    true_last_number = extract_number(true_last.split('/')[-2])
    pages = [true_last.replace(str(true_last_number), str(i)) for i in range(1, true_last_number + 1)]
    
    all_pages_houses.append(pages)

all_pages_houses = flatten(all_pages_houses)

## Product Links

In [ ]:
all_products_apartments = []

for page in tqdm(all_pages_apartments):
    header = {'User-Agent':str(ua.random)}
    content = requests.get(page, headers = header)
    soup = BeautifulSoup(content.content, 'html.parser')

    product_container = soup.find('ul', {'class':'itemlist ad-list it3'})
    if product_container is not None:
        product_container_narrow = product_container.find_all('li', {'class':'ad-listitem'})
        products_raw = [i.find('article', {'class':'aditem'}) for i in product_container_narrow]

        all_products = []
        for i in products_raw:
            if i != None:
                products = mother_url + i.find('a', {'href':True})['href']
                all_products.append(products)
            else:
                continue
        all_products_apartments.append(all_products)
    else:
        continue
    
all_products_apartments = uniquize(flatten(all_products_apartments))

In [ ]:
all_products_houses = []

for page in tqdm(all_pages_houses):
    header = {'User-Agent':str(ua.random)}
    content = requests.get(page, headers = header)
    soup = BeautifulSoup(content.content, 'html.parser')

    product_container = soup.find('ul', {'class':'itemlist ad-list it3'})
    if product_container is not None:
        product_container_narrow = product_container.find_all('li', {'class':'ad-listitem'})
        products_raw = [i.find('article', {'class':'aditem'}) for i in product_container_narrow]

        all_products = []
        for i in products_raw:
            if i != None:
                products = mother_url + i.find('a', {'href':True})['href']
                all_products.append(products)
            else:
                continue
        all_products_houses.append(all_products)
    else:
        continue
    
all_products_houses = uniquize(flatten(all_products_houses))

## Data Collection

### Apartments

In [76]:
# all_sizes = []
# all_links1 = []
# all_rooms1 = []
# all_final_costs1 = []
# all_zip_codes1 = []
# all_latitudes1 = []
# all_longitudes1 = []
# all_floors1 = []
# all_balconies = []
# all_storages = []
# all_elevators = []


for link in tqdm(all_products_apartments[1349+3637+45+349:]):
    header = {'User-Agent':str(ua.random)}
    content = requests.get(link, headers = header)
    soup = BeautifulSoup(content.content, 'html.parser')
    
    test_price = soup.find('h2', {'class':'boxedarticle--price'})
    if test_price:
        price_raw = test_price.text.strip().replace('€', '').replace('.', '').replace(',', '.')
        if has_number(price_raw):
            price = float(test_price.text.strip().replace('€', '').replace('.', '').replace(',', '.').replace('  VB', ''))
        else:
            price = None
    else:
        price = None

    test_address = soup.find('div', {'itemprop':'address'})
    if test_address:
        address_container = test_address.find_all('span')
        address_raw = [i.text.strip().replace(',', '') for i in address_container]
        address_raw = flatten([i.split() for i in address_raw])

        zip_code = None
        for i in address_raw:
            if contains_only_numbers(i) and len(i) == 5:
                zip_code = int(i)
            else:
                continue
        try:
            latlong = addressToCoord(' '.join(address_raw))
        except:
            latlong = None

        if latlong != None:    
            latitude = float(latlong[0])
            longitude = float(latlong[1])
        else:
            latitude = None
            longitude = None
    else:
        latitude = None
        longitude = None
        zip_code = None
        
    test_info = soup.find('ul', {'class':'addetailslist'})
    if test_info:
        info_container = test_info.find_all('li', {'class':'addetailslist--detail'})
        info_raw = [i.text.replace('\n                                       ', '').strip().lower() for i in info_container]

        interior_size = None
        rooms = None
        floor = None
        for component in info_raw:
            if 'wohnfläche' in component:
                interior_size = float(component.replace('wohnfläche ', '').replace(' m²', '').replace('.', '').replace(',', '.'))
            elif 'zimmer' in component and 'bade' not in component and 'schlaf' not in component and 'ess' not in component and 'privat' not in component and 'kinder' not in component:
                rooms = float(component.replace('zimmer ', '').replace('.', '').replace(',', '.'))
            elif 'etage' in component and 'wohnung' not in component or 'etagen' in component and 'etagenwohnung' not in component:
                floor = int(component.replace('etage ', '').replace('etagen ', ''))
            else:
                continue
    else:
        rooms = None
        interior_size = None
        floor = None

    test_additionals = soup.find('ul', {'class':'checktaglist'})
    if test_additionals:
        additionals_container = test_additionals.find_all('li', {'class':'checktag'})
        additionals_raw = [i.text.strip().lower() for i in additionals_container]

        balcony = 0
        storage = 0
        elevator = 0
        for i in additionals_raw:
            if 'balkon' in i or 'terrasse' in i:
                balcony = 1
            elif 'keller' in i:
                storage = 1
            elif 'aufzug' in i:
                elevator = 1
            else:
                continue
    else:
        balcony = 0
        storage = 0
        elevator = 0
                
    all_sizes.append(interior_size)
    all_rooms1.append(rooms)
    all_final_costs1.append(price)
    all_zip_codes1.append(zip_code)
    all_latitudes1.append(latitude)
    all_longitudes1.append(longitude)
    all_floors1.append(floor)
    all_balconies.append(balcony)
    all_storages.append(storage)
    all_elevators.append(elevator)
    all_links1.append(link)
    
all_sizes = flatten(all_sizes)
all_rooms1 = flatten(all_rooms1)
all_final_costs1 = flatten(all_final_costs1)
all_zip_codes1 = flatten(all_zip_codes1)
all_latitudes1 = flatten(all_latitudes1)
all_longitudes1 = flatten(all_longitudes1)
all_floors1 = flatten(all_floors1)
all_balconies = flatten(all_balconies)
all_storages = flatten(all_storages)
all_elevators = flatten(all_elevators)
all_links1 = flatten(all_links1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.


### Houses

In [55]:
all_interior_sizes = []
all_exterior_sizes = []
all_links2 = []
all_rooms2 = []
all_final_costs2 = []
all_zip_codes2 = []
all_latitudes2 = []
all_longitudes2 = []
all_floors2 = []
all_parkings = []
all_house_types = []
all_terraces = []
all_years = []

for link in tqdm(all_products_houses):
    header = {'User-Agent':str(ua.random)}
    content = requests.get(link, headers = header)
    soup = BeautifulSoup(content.content, 'html.parser')
    
    test_price = soup.find('h2', {'class':'boxedarticle--price'})
    if test_price:
        price_raw = test_price.text.strip().replace('€', '').replace('.', '').replace(',', '.')
        if has_number(price_raw):
            price = float(test_price.text.strip().replace('€', '').replace('.', '').replace(',', '.').replace('  VB', ''))
        else:
            price = None
    else:
        price = None

    test_address = soup.find('div', {'itemprop':'address'})
    if test_address:
        address_container = test_address.find_all('span')
        address_raw = [i.text.strip().replace(',', '') for i in address_container]
        address_raw = flatten([i.split() for i in address_raw])

        zip_code = None
        for i in address_raw:
            if contains_only_numbers(i) and len(i) == 5:
                zip_code = int(i)
            else:
                continue
        try:
            latlong = addressToCoord(' '.join(address_raw))
        except:
            latlong = None

        if latlong != None:    
            latitude = float(latlong[0])
            longitude = float(latlong[1])
        else:
            latitude = None
            longitude = None
    else:
        latitude = None
        longitude = None
        zip_code = None

    test_info = soup.find('ul', {'class':'addetailslist'})
    if test_info:
        info_container = test_info.find_all('li', {'class':'addetailslist--detail'})
        info_raw = [i.text.replace('\n                                       ', '').strip().lower() for i in info_container]

        interior_size = None
        exterior_size = None
        rooms = None
        floor = None
        house_type = None
        year = None
        for component in info_raw:
            if 'wohnfläche' in component and 'grund' not in component:
                interior_size = float(component.replace('wohnfläche ', '').replace(' m²', '').replace('.', '').replace(',', '.'))
            elif 'zimmer' in component and 'bade' not in component and 'schlaf' not in component  and 'ess' not in component:
                rooms = float(component.replace('zimmer ', '').replace('.', '').replace(',', '.'))
            elif 'grundstücksfläche' in component and 'wohn' not in component:
                exterior_size = float(component.replace('grundstücksfläche ', '').replace(' m²', '').replace('.', '').replace(',', '.'))    
            elif 'etagen' in component and 'wohnung' not in component:
                floor = int(component.replace('etagen ', ''))
            elif 'haustyp' in component:
                house_type = GoogleTranslator(source='de', target='en').translate(component.replace('haustyp ', '')).lower()
            elif 'baujahr' in component:
                year = int(component.replace('baujahr ', ''))
            else:
                continue
    else:
        rooms = None
        interior_size = None
        exterior_size = None
        floor = None
        house_type = None
        year = None

    test_additionals = soup.find('ul', {'class':'checktaglist'})
    if test_additionals:
        additionals_container = test_additionals.find_all('li', {'class':'checktag'})
        additionals_raw = [i.text.strip().lower() for i in additionals_container]

        terrace = 0
        parking = 0
        for i in additionals_raw:
            if 'balkon' in i or 'terrasse' in i:
                terrace = 1
            elif 'garage' in i or 'stellplatz' in i:
                parking = 1
            else:
                continue
    else:
        terrace = 0
        parking = 0
                
    all_interior_sizes.append(interior_size)
    all_exterior_sizes.append(exterior_size)
    all_rooms2.append(rooms)
    all_final_costs2.append(price)
    all_zip_codes2.append(zip_code)
    all_latitudes2.append(latitude)
    all_longitudes2.append(longitude)
    all_floors2.append(floor)
    all_links2.append(link)
    all_parkings.append(parking)
    all_house_types.append(house_type)
    all_terraces.append(terrace)
    all_years.append(year)
    
all_interior_sizes = flatten(all_interior_sizes)
all_exterior_sizes = flatten(all_exterior_sizes)
all_rooms2 = flatten(all_rooms2)
all_final_costs2 = flatten(all_final_costs2)
all_zip_codes2 = flatten(all_zip_codes2)
all_latitudes2 = flatten(all_latitudes2)
all_longitudes2 = flatten(all_longitudes2)
all_floors2 = flatten(all_floors2)
all_house_types = flatten(all_house_types)
all_parkings = flatten(all_parkings)
all_links2 = flatten(all_links2)
all_terraces = flatten(all_terraces)
all_years = flatten(all_years)

# Storage

In [82]:
data_apartments = pd.DataFrame()
data_houses = pd.DataFrame()

data_apartments['Link'] = all_links1
data_apartments['Interior Size'] = all_sizes
data_apartments['Room Count'] = all_rooms1
data_apartments['Zip Code'] = all_zip_codes1
data_apartments['Latitude'] = all_latitudes1
data_apartments['Longitude'] = all_longitudes1
data_apartments['Floor Level'] = all_floors1
data_apartments['Balcony'] = all_balconies
data_apartments['Storage'] = all_storages
data_apartments['Elevator'] = all_elevators
data_apartments['Price'] = all_final_costs1
data_apartments['Zip Code'] = data_apartments['Zip Code'].fillna(0).astype(int).replace(0, None)

data_houses['Link'] = all_links2
data_houses['House Type'] = all_house_types
data_houses['Year Constructed'] = all_years
data_houses['Interior Size'] = all_interior_sizes
data_houses['Exterior Size'] = all_exterior_sizes
data_houses['Room Count'] = all_rooms2
data_houses['Zip Code'] = all_zip_codes2
data_houses['Latitude'] = all_latitudes2
data_houses['Longitude'] = all_longitudes2
data_houses['Floor Count'] = all_floors2
data_houses['Parking'] = all_parkings
data_houses['Terrace'] = all_terraces
data_houses['Price'] = all_final_costs2
data_houses['Zip Code'] = data_houses['Zip Code'].fillna(0).astype(int).replace(0, None)

In [83]:
def format_value(value):
    if value >= 1000 and value <= 9999:
        return f'0{value:04}'
    return str(value)

data_apartments['Zip Code'] = data_apartments['Zip Code'].apply(format_value)
data_houses['Zip Code'] = data_houses['Zip Code'].apply(format_value)

In [84]:
data_apartments.to_excel('kleineinzeigenApartments.xlsx', index=False, engine='openpyxl')
data_houses.to_excel('kleineinzeigenHouses.xlsx', index=False, engine='openpyxl')

In [67]:
data_apartments = pd.read_excel('kleineinzeigenApartments.xlsx', engine='openpyxl', dtype={'Zip Code': str})
data_houses = pd.read_excel('kleineinzeigenHouses.xlsx', engine='openpyxl', dtype={'Zip Code': str})